### Conversational Chatbot

Connect your google drive to collab here

#### why?

- I did it so i dont have to keep on uploading the test file
- Also i didnt want to downloading INtructor embedding and other cached files on every rerun.
- As you guessed i cannot run the models locally

Navigating to where my stuff is

In [ ]:
%cd drive/My Drive/gdrive

Now, i was curious about how my code was performing aka how much load the system was in at what times, hence i was using this to acces the virtual machines terminal

In [ ]:
!pip install colab-xterm
%load_ext colabxterm
%xterm

#apt-get install glances

### PIP and import
Remember to require htmltemps and slck

In [ ]:
!pip install streamlit PyPDF2 langchain slack_sdk transformers faiss-gpu InstructorEmbedding


In [ ]:
#! git clone https://github.com/HKUNLP/instructor-embedding
!pip install -r /content/drive/MyDrive/gdrive/instructor-embedding/requirements.txt

While most of whats here is self explanatory allow me to provide additional information

-- Slick is a personal file i created, what that program does is, it sends me a message on slack when my code is done executing, since it was taking quite a while, i figured this way i can focus on other tasks

In [ ]:
import streamlit as st
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import HuggingFaceHub
from htmlTemplates import css, bot_template, user_template
import torch
from slick import pingd,pingdm
from tqdm import tqdm
from numba import cuda

### Method

In the method i have chosen to run my web app, i dont really need to run these cells. But i do anyway so it downloads and caches the model when i finally run the main code



First and foremost is the loading of the pdf file that needs to be read and then break it into fair chunks, i have also taken the liberty to clear up gpu memory as a python notebook is not the way to go when working with such heavy memory models

In [ ]:
def get_pdf_text():
    pdf_docs = ['/content/drive/MyDrive/gdrive/hib2.pdf']
    text=''
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=500,
        chunk_overlap=200,
        length_function=len
    )

    chunks = text_splitter.split_text(text)
    return chunks

def release():
    cuda.select_device(0)
    cuda.close()

In my program i have given multiple models and embedding methods that you can choose to acheive your desired performance levels, the draw back of course is that you have to initialise all of it

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

Here we create a database or a vectorstore using our desired embedding method
This is so its easier for the model to find and match our answer

In [ ]:
def get_vectorstore():
    #embeddings = OpenAIEmbeddings()

    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

    #text_chunks = tqdm(text_chunks)

    #vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    #pingd()
    #vectorstore.save_local("faiss_index")
    vectorstore = FAISS.load_local("faiss_index", embeddings)
    return vectorstore

vectorstore = get_vectorstore()


Insert your personal hugging face api token 
(also keep in mind at the time i used llama, it was a gated repo, so you will also proabbly have to ask for permission)

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'hf_xyz'

The most important part of the program, create a pipeline for the model and create a conversation chain, whether you run it personally or on hugging face, i still needed to wrap it in a certain way to be able to use it with langchain.

Here i also insert input variables because i want my program to have past chat references, so it can take in relational questions better

In [ ]:
def get_conversation_chain(vectorstore):
    #llm = ChatOpenAI()
    llm = HuggingFaceHub(repo_id="meta-llama/Llama-2-7b-chat-hf", model_kwargs={"temperature":0.5, "max_length":512,"min_length":100})
    #llm = HuggingFaceHub(repo_id="eachadea/ggml-vicuna-13b-1.1", model_kwargs={"temperature":0.4, "max_length":1024,"min_length":100})
    template = """The following is an informative conversation between a user and a bot.The bot will provide answers to the user from the context of health insurance brochures.The bot is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

    Current conversation:
    {history}
    user: {input}
    bot:"""
    PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

x=get_conversation_chain(vectorstore)

force login

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

This is purely for testing purposes i wanted to clear up whether the chain created was performing as intended

In [ ]:
print(x("Explain in detail include all points if possible, Keep it to the point, Question: Special features of health infinity"))

As the name suggests it handles user input , it sends the question and the chat history to the chain as a query.


It also prints the response beautifully

In [ ]:
def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
            print(user_template.replace("{{MSG}}", message.content))
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
            print(bot_template.replace("{{MSG}}", message.content))

handle_userinput2("ambulance policy")


### Main

Here i am using streamlit to create a wonderful UI to interact with my Conversational bot.
A few things to point out here are
- It retains history and relation while having a conversation
- Runs all the functions defined above in a particular order
- And looks good when it does that


In [ ]:
def main():
    st.set_page_config(page_title = "Chat with books cuz you are lonely",page_icon=":books:")
    st.write(css, unsafe_allow_html=True)

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None

    st.header("Lets begin cuties :purple_heart:")
    user_question = st.text_input("Go on, ask me anything")
    if user_question:
        handle_userinput(user_question)



    st.write(user_template.replace("{{MSG}}", "Waiting on you"), unsafe_allow_html=True)
    with st.sidebar:
        st.subheader("We dont have all day")
        pdf_docs = st.file_uploader("Documents",accept_multiple_files=True)
        if st.button("Process"):
            with st.spinner("Processing"):
                # get pdf text
                raw_text = get_pdf_text(pdf_docs)

                # get the text chunks
                text_chunks = get_text_chunks(raw_text)


                # create vector store
                vectorstore = get_vectorstore(text_chunks)
                print("returned vectorizer")

                # create conversation chain
                st.session_state.conversation = get_conversation_chain(vectorstore)
                print("Begin")


### Streamlit Stuff

Streamlit is a great way to have a quick and easy UI to try out chatbots. However its slightly tricky to do so in a google collab, since you cant exactly have a local instance

#### I have used this method to overcome that

- First run the below code to get your vm's Ip address
- Run the next cell so streamlit opens the app you previously wrote and it does it with the help of npx tunnel that temporarily assigns you a webpage that behaves as your localhost

In [ ]:

!wget -q -O - ipv4.icanhazip.com


34.80.56.191


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

### The App

For the sake of understanding, we discussed a simpler version of this App. The changes that this has compared to above is

- Added options to try multiple models and embedding techniques to see which one works best for you
- Optimised it so it doesnt break down as often (especially if your system is having a tough time)
- (Much) Better error handling
- UI looks even better

The first line writefile app.py allows you to write this into a file that streamlit will then use

In [ ]:
%%writefile app.py

from torch import cuda
from huggingface_hub import login
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.embeddings import GPT4AllEmbeddings
from llama_index.embeddings import LangchainEmbedding

import tiktoken
from PyPDF2 import PdfReader
from transformers import AutoTokenizer
import transformers
import torch
from transformers import StoppingCriteria, StoppingCriteriaList
from langchain import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
import streamlit as st
from langchain.embeddings import OpenAIEmbeddings,HuggingFaceInstructEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFaceHub
from langchain.prompts.prompt import PromptTemplate
from langchain.chat_models import ChatOpenAI
import torch
import time

#personal additions
# from slick import pingdm,pingd,internexists
from htmlTemplates import css, bot_template, user_template

err = "No errors"


# Joke
# try:
#     if internexists():
#         pass
#     else:
#         raise Exception('Whoopsie daisy') 
# except:
#     err = "Code's free trial has expired"
#     quit()
# else: 
    # name = "meta-llama/Llama-2-7b-chat-hf"
    # auth_token = "hf_xyz"
    # openai_api_key="sk-xyz"


#Insert your own
name = "meta-llama/Llama-2-7b-chat-hf"
auth_token = "hf_xyz"
openai_api_key="sk-xyz"

##Small conviniences Codes
embc = {1: "OpenAi [#Paid Fastest]", 2: "Instructor [#Rank 1 Slow]", 3: "Minim [#Rank 4 Fast]", 4:"GPT4All [#Rank 3 Balanced]"}
embm = {1: "Llama 2 7b", 2: "Open Ai", 3: "Eleuther",4:"Flan T5",5:"Falcon"}


def format_func(option, data):
    return data.get(option, "Unknown")

class StopOnTokens(StoppingCriteria):
    def __init__(self, stop_ids):
        self.stop_ids = stop_ids

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in self.stop_ids:
            if torch.eq(input_ids[0][-len(stop_id):], stop_id).all():
                return True
        return False



def get_pdf_text(pdf_docs):
    text = ""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=600,
        chunk_overlap=160,
        length_function =len
    )

    chunks = text_splitter.split_text(text)
    return chunks


def load_vectorstore(x,pdf_docs):
    global err
    err = "Reached Vector manager"

    if x==1:
        embeddings = OpenAIEmbeddings(openai_api_key)
        try:
            vectorstore = FAISS.load_local("emb/faiss_index ai", embeddings)
            err = "Open Ai : Save found"
        except:
            try:
                text_chunks = get_text_chunks(get_pdf_text(pdf_docs))
                vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
                vectorstore.save_local("emb/faiss_index ai")
            except:
                err = "This failed? Strange... Trace: load vectorstore if else"


    elif x==3 or x==2:
        # Create and dl embeddings instance  
        embeddings=LangchainEmbedding(HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"))
        try:
            vectorstore = FAISS.load_local("emb/faiss_index mi", embeddings)
            err = "Minim : Save found"
        except:
            try:
                text_chunks = get_text_chunks(get_pdf_text(pdf_docs))
                vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
                vectorstore.save_local("emb/faiss_index mi")
            except:
                err = "Bro how can you mess up twice?? Trace: load vectorstore if else"

    
    elif x==4:
        embeddings = GPT4AllEmbeddings()
        try:
            vectorstore = FAISS.load_local("emb/faiss_index g4", embeddings)
            err = "GPT4All : Save found"
        except:
            try:
                text_chunks = get_text_chunks(get_pdf_text(pdf_docs))
                vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
            except:
                err = "GPT4All Trace: load vectorstore if else"
        pass

    elif x==21:
        #Verbose disabled till i am done sorting the model directory
        st.text_area("Disabled till i am done sorting the program", height=50, disabled=True)
        
        embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
        try:
            vectorstore = FAISS.load_local("emb/faiss_index hi", embeddings)
        except:
            err = "in progress"
    print("#####",err,"#####")
    print(type(vectorstore))
    return vectorstore



def initialize():
    tokenizer = AutoTokenizer.from_pretrained(name)
    stop_list = ['\nHuman:', '\n```\n']
    stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
    device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
    print("Stopping Criteria")
    stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
    stopping_criteria = StoppingCriteriaList([StopOnTokens(stop_token_ids)])
    print("Passed Criteria on to pipeline")

    pipeline = transformers.pipeline(
        task="text-generation", #task
        model=name,
        tokenizer=tokenizer,
        return_full_text=True,  # langchain expects the full text
        stopping_criteria=stopping_criteria,  # without this model rambles during chat
        temperature=0.2,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
        repetition_penalty=1.1,  # without this output begins repeating
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
        max_length=4000,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id
        )
    return pipeline


def get_conversation_chain(vectorstore,optm,pipeline=None):
    global err
    print("Reached LLM")
    err = "Reached LLM with the option",optm
    if pipeline==None:
        if optm == 2:
            llm = ChatOpenAI(openai_api_key)
            err = "The OpenAi route"

        elif optm == 4:
            llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})
            err = "Loaded Flan T5 from HuggingFaceHub"
        
        elif optm == 5:
            
            pass

        else:
            err = "Llama not chosen"
            return err
        
    else:
        err = "Moving with LLama 🦙"
        llm = HuggingFacePipeline(pipeline = pipeline)

    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    retriever = vectorstore.as_retriever()

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        memory=memory
    )

    return conversation_chain

def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)


def main():
    login(token=auth_token)
    st.set_page_config(page_title = "Chat with books cuz you are lonely",page_icon=":books:")
    st.write(css, unsafe_allow_html=True)

    if "conversation" not in st.session_state:
        st.session_state.conversation = None
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = None
    if "message" not in st.session_state:
        st.session_state.message = None


    st.header("Lets begin cuties :purple_heart:")
    user_question = st.text_input("Go on, ask me anything")
    if user_question:
        handle_userinput(user_question)

    
    init = st.empty()
    init.write(user_template.replace("{{MSG}}", "Waiting on you"), unsafe_allow_html=True)

    with st.sidebar:

        st.subheader("Choice of Embedding")
        opte = st.selectbox(label="Embeddings",placeholder="Choose?", options=list(embc.keys()), format_func=lambda x: format_func(x, embc))

        st.subheader("Choose your Model")
        optm = st.selectbox(label="Models",placeholder="Choose?", options=list(embm.keys()), format_func=lambda x: format_func(x, embm))

        st.subheader("We dont have all day")
        pdf_docs = st.file_uploader("Documents",accept_multiple_files=True)
        
        if st.button("Process"):
            with st.spinner("Processing"):

                #Handle files and load vectorstore
                stt = time.time()
                vectorstore = load_vectorstore(opte,pdf_docs)
                ett=time.time()
                timex=ett-stt

                print("Successfully returned vectorizer")
                init.write(user_template.replace("{{MSG}}", str(err)), unsafe_allow_html=True)
                pingd(err,str(timex)[:5])
                
                if optm==1:
                    pipeline = initialize()
                    print("Pipeline initialized")
                    pingd("Pipeline initialized")

                # create conversation chain
                st.session_state.conversation = get_conversation_chain(vectorstore,optm,pipeline)
                print("Begin")
                init.write(user_template.replace("{{MSG}}","Ask me a Question ?"), unsafe_allow_html=True)



if __name__ == '__main__':
    main()


# st.error("Do you really, really, wanna do this?")
# if st.button("Yes I'm ready to rumble"):
#     run_expensive_function()